In [526]:
import pandas as pd
import pickle
import numpy as np
from sklearn.cluster import KMeans
%matplotlib inline

In [394]:
def metrics(p, t):
    return 200 * pd.DataFrame(np.fabs(p - t) / (np.fabs(p) + np.fabs(t))).mean()[0]

In [564]:
with open('./data/data_with_date2.p', 'rb') as f:
    df_ = pickle.load(f)

In [ ]:
df_['date'] = df_['DATET']
df_.drop('DATET', inplace=True, axis=1)

In [576]:
df_["day"] = df_.date.dt.day
df_["dayofweek"] = df_.date.dt.dayofweek
df_["is_month_end"] = df_.date.dt.is_month_end.astype(int)
df_["is_month_start"] = df_.date.dt.is_month_start.astype(int)
df_["week"] = df_.date.dt.week
df_["weekend"] = (df_.date.dt.dayofweek>=5).astype(int)

In [232]:
'''
Для каждой АЗС посчитать цены на бензин и посчитать персентили

'''

'\nДля каждой АЗС посчитать цены на бензин и посчитать персентили\n\n'

In [873]:
df_train = df_[
#     (df_.date >= pd.Timestamp('20171230') - pd.Timedelta("220 day")) &
    (df_.date <= pd.Timestamp('20171230') - pd.Timedelta("90 day"))
].copy()

In [847]:
df_test = df_[
    (df_.date > pd.Timestamp('20171230') - pd.Timedelta("90 day"))
].copy()

In [877]:
df_test = pd.read_csv("./data/test_data.csv")
df_test_full = pd.read_csv("./data/full_test_data.csv")
df_train = df_.copy()

/usr/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [878]:
gfields = ["REGION_AZS", "VID_NP", "CLIENT", "AZS_NUMBER"]#"REGION_AZS", 'AZS_NUMBER', "REGION_AZS", 

df_res = pd.DataFrame(columns=['ID', 'CLIENT', 'COL_LITR'])
def get_pred(df_train, df_test, fields):
    r = df_train.groupby(fields).agg({"COL_LITR":'median', "ID":"count"})
    r.reset_index(inplace=True)
    df_result = df_test[['ID'] + fields].merge(r[['COL_LITR'] + fields], how="left", on=fields)
    
    return df_result

# df_train_ = df_train.copy()
df_test_ = df_test.copy()

for i in range(len(gfields)):
    fields = gfields[:len(gfields)-i]
    
    result = get_pred(df_train, df_test_, fields)
    
    idx_na = result[result.COL_LITR.isnull()].index
    idx = result[result.COL_LITR.isnull() == False].index
    
    print(idx_na.shape, idx.shape)
    
    df_res = pd.concat([df_res, result.iloc[idx][['ID', 'COL_LITR']]], axis=0)#, 'CLIENT'
    
    df_test_ = df_test_.iloc[idx_na].copy()


(209828,) (790172,)
(154071,) (55757,)
(1041,) (153030,)
(1041,) (0,)


In [879]:
predict = df_test[['ID']].merge(df_res, how='left', on="ID")

In [830]:
predict['true'] = df_test['COL_LITR'].values
metrics(predict['COL_LITR'].fillna(40).values, predict['true'].values)

39.18509396048809

In [880]:
df_test_target = df_test_full.copy()
# df_test_target = pd.read_csv("./data/full_test_data.csv")
df_test_target['COL_LITR'] = 40
X_tst, y_tst = train_xgb(df_test_target)

In [882]:
X_tst['COL_LITR'] = regressor.predict(X_tst.drop(['CLIENT', 'AZS_NUMBER'], axis=1))

In [883]:
df_prepare = df_test[["CLIENT", "VID_NP","REGION_AZS", "AZS_NUMBER", "ID"]].merge(X_tst[["CLIENT", "VID_NP","REGION_AZS", "AZS_NUMBER", "COL_LITR"]], how='left', on=["CLIENT", "VID_NP","REGION_AZS", "AZS_NUMBER"])

In [884]:
df_pred_merge = pd.DataFrame(predict[predict.COL_LITR.isnull()]['ID']).\
    merge(df_prepare[['ID', 'COL_LITR']], how='left', on='ID')

In [885]:
predict["COL_LITR"][predict.COL_LITR.isnull()]=\
    predict[predict.COL_LITR.isnull()].ID.map(df_pred_merge.set_index("ID").COL_LITR)

/usr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [653]:
metrics(predict['COL_LITR'].values, predict['true'].values)
# metrics(np.round(predict.fillna(40)['COL_LITR'].values), predict['true'].values)

38.01786940847149

In [886]:
predict[['ID', 'COL_LITR']].to_csv("./submissions/kolesnikov_mk8.csv", index=False)

In [887]:
print("DONE!!!!")

DONE!!!!


In [874]:
df_train_xgb = df_[
    (df_.date >= pd.Timestamp('20171230') - pd.Timedelta("270 day")) &
    (df_.date <= pd.Timestamp('20171230') - pd.Timedelta("90 day"))
].copy()

In [832]:
def train_xgb(df_train_xgb):
    df_region = df_train_xgb.groupby(['REGION_AZS']).agg({
        "ID":"count", "CLIENT": "nunique", "AZS_NUMBER":"nunique", "CENA_CLIENT":"median"
    })
    df_region = df_region.reset_index()
    df_region.columns = ['REGION_AZS', 'REGION_COUNT', 'REGION_CLIENT_U', 'REGION_AZS_U', 'REGION_CENA_M']
    
    df_azs = df_train_xgb.groupby(['AZS_NUMBER']).agg({
        "ID":"count", "CLIENT": "nunique", "CENA_CLIENT":{"nunique", "median"}
    })
    df_azs = df_azs.reset_index()
    df_azs.columns = ['AZS_NUMBER', 'AZS_COUNT', 'AZS_CLIENT_U', 'AZS_CENA_CLIENT_M', 'AZS_CENA_CLIENT_U']
    
    df_client = df_train_xgb.groupby(['CLIENT']).agg({
        "ID":"count", "AZS_NUMBER": "nunique", "CENA_CLIENT":{"nunique", "median"}, "REGION_AZS":"nunique"
    })
    df_client.columns = ['CLIENT_ID_COUNT', 'CLIENT_AZS_NUMBER_U', 'CLIENT_CENA_CLIENT_U', 'CLIENT_CENA_CLIENT_M', 'CLIENT_REGION_AZS_U']
    df_client = df_client.reset_index()
    
    df_xgb = df_train_xgb.groupby(["CLIENT", "VID_NP","REGION_AZS", "AZS_NUMBER"]).agg({'COL_LITR':'median'})
    df_xgb = df_xgb.reset_index()
    
    df_xgb = df_xgb.merge(df_region, how="left", left_on='REGION_AZS', right_on='REGION_AZS')
    df_xgb = df_xgb.merge(df_azs, how="left", left_on='AZS_NUMBER', right_on='AZS_NUMBER')
    df_xgb = df_xgb.merge(df_client, how="left", left_on='CLIENT', right_on='CLIENT')
    
    X = df_xgb.drop(['COL_LITR'], axis=1)#'CLIENT', 'AZS_NUMBER', 
    y = df_xgb[['COL_LITR']]
    
    return X, y

In [845]:
X_train, y_train = train_xgb(df_train_xgb)

In [848]:
X_test, y_test = train_xgb(df_test)

In [849]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

In [875]:
# regressor = RandomForestRegressor(n_estimators=200, n_jobs=30, verbose=1, max_depth=8)
regressor = XGBRegressor(nthread=60, seed=2018, n_estimators=300, learning_rate=0.01, max_depth=10)

In [876]:
%%time
regressor.fit(X_train.drop(['CLIENT', 'AZS_NUMBER'], axis=1), y_train)

CPU times: user 46min 23s, sys: 7.84 s, total: 46min 31s
Wall time: 2min


XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.01, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=300, nthread=60,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=2018, silent=True, subsample=1)

In [867]:
xgb_predict = regressor.predict(X_test.drop(['CLIENT', 'AZS_NUMBER'], axis=1))

In [868]:
ppred = regressor.predict(X_test.drop(['CLIENT', 'AZS_NUMBER'], axis=1))
yy_test = [x[0] for x in y_test.as_matrix()]
metrics(np.array(ppred), np.array(yy_test))
# np.fabs(np.array(preditc) - np.array(y_test)) / (np.fabs(np.array(preditc)) + np.fabs(np.array(y_test))).mean()

# np.fabs(np.array(preditc) - y_test.values)
# yy_test

43.940070884037254

In [814]:
df_test_pred = X_test[["CLIENT", "VID_NP","REGION_AZS", "AZS_NUMBER"]]
df_test_pred['COL_LITR'] = xgb_predict

/usr/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [805]:
with open('./kolesnikov_reg_2.pkl', 'wb') as f:
    pickle.dump(regressor, f)